# Многопроцессорность

Потоки существуют внутри одного процесса и работают с общими ресурсами этого процесса. Однако у процессов, по сути, своя область памяти и свой внутренний GIL (Global Interpreter Lock). То есть, когда мы говорим о потоках, мы говорим о конкурентном выполнении кода. Когда мы говорим о процессах, мы говорим о параллельном выполнении на разных ядрах вашего процессора. Однако из-за этого программа становится тяжелее для компьютера. И так как у каждого процесса своя область памяти, соответственно, сложнее становится их координировать между собой.

In [1]:

from threading import Thread
import time


counter = 0

def first_worker(n):
    global counter
    for i in range(n):
        counter += 1
        time.sleep(1)
    print('Первый рабочий изменил значение счётчика на ', counter)

def second_worker(n):
    global counter
    for i in range(n):
        counter += 1
        time.sleep(1)
    print('Второй рабочий изменил значение счётчика на ', counter)

t1 = Thread(target=first_worker, args=(15,))
t2 = Thread(target=second_worker, args=(10,))

t1.start()
t2.start()

t1.join()
t2.join()
print('Итоговое значение счётчика:', counter)



Второй рабочий изменил значение счётчика на  20
Первый рабочий изменил значение счётчика на  25
Итоговое значение счётчика: 25


Но что, если мы поменяем «thread» на «process». В плане написания процессы ничем не отличаются от потоков, мы можем применять все те же самые методы. Мы можем использовать два способа для создания процессов: либо использовать уже готовый класс «Process» и передавать туда в «target» нужную функцию и в «args» параметры нашей функции, либо создать класс и переопределить в нём, например, метод «run», если нам нужна какая-то особая логика поведения. Также мы можем проверять, живы ли процессы, и принудительно останавливать их. В этом они очень схожи с потоками.

Однако, если мы здесь напишем «process1» и обратимся к нашему модулю «multiprocessing», создадим объект класса «Process», где в качестве «target» укажем нашу первую функцию и передадим «args» со значением 10. Для второго процесса укажем в качестве «target» вторую функцию и передадим, например, 15. Запустим оба процесса, и при таком написании мы получим ошибку

In [3]:
from multiprocessing import Process
counter = 0

t1 = Process(target=first_worker, args=(15,))
t2 = Process(target=second_worker, args=(10,))

t1.start()
t2.start()

print('Итоговое значение счётчика:', counter)


Итоговое значение счётчика: 0


Чтобы избавиться от этой ошибки, нам нужно использовать конструкцию «if __name__ == '__main__':». При таком раскладе у нас всё отлично отрабатывает 

In [ ]:
if __name__ == '__main__':
    t1 = Process(target=first_worker, args=(15,))
    t2 = Process(target=second_worker, args=(10,))

    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print('Итоговое значение счётчика:', counter)

Однако сейчас мы ожидаем получить результат для первого рабочего 15, для второго рабочего 10. То есть у каждого рабочего своя область памяти.

По сути мы создали несколько экземпляров нашей программы, которые выполняют определённую функция, и они выполняются параллельно. В этом мы можем убедиться, если будем выводить значение нашей переменной «counter». В результате запуска мы видим, что нет никаких наслоений, «print» и значения идентичные, что в случае работы одного процесса, что в случае работы второго процесса

# Составление связи между процессами

# Очереди(Queue)

Очереди являются одним из самых простых и безопасных способов обмена данными между процессами. Они позволяют передавать данные от одного процесса к другому.

In [4]:
from multiprocessing import Process, Queue

def worker(q):
    data = q.get()
    print(f'Process1 received: {data}')

def worker2(q):
    data = q.get()
    print(f'Process2 received: {data}')
    
if __name__ == '__main__':
    q = Queue()
    p = Process(target=worker, args=(q, ))
    p2 = Process(target=worker2, args=(q, ))
    p.start()
    p2.start()
    q.put('Hello from main process')
    q.put('Hello')
    p.join()
    p2.join()


# Каналы(Pipe)

Каналы предоставляют двустороннюю связь между процессами. Они полезны, когда нужно организовать обмен данными в обоих направлениях.

In [5]:
from multiprocessing import Process, Pipe
from multiprocessing.connection import PipeConnection

def worker(conn: PipeConnection):
    data = conn.recv() # получаем данные из канала
    print('Process received: {data}')
    conn.send('Hello from worker')
    
if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(targar=worker, args=(child_conn, ))
    p.start()
    
    parent_conn.send('Hello from main process')
    print(parent_conn.recv()) # получаем ответ от процесса
    p.join()


TypeError: BaseProcess.__init__() got an unexpected keyword argument 'targar'. Did you mean 'target'?